In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

import tweepy

# Your credentials
bearer_token = "AAAAAAAAAAAAAAAAAAAAAOx75QEAAAAAu%2Fg1DMlRQR6Bs16xyxwcHdV53nA%3De2srLerE5MMpvVFyaO1JnVhY3cQ0QAGIYXl2lFAGmd3QxlR6yS"

# Authenticate
client = tweepy.Client(bearer_token=bearer_token)

# Example: Search recent tweets about Bitcoin
query = "bitcoin lang:en -is:retweet"
tweets = client.search_recent_tweets(query=query, max_results=10)

for tweet in tweets.data:
    print(tweet.text)


ModuleNotFoundError: No module named 'tweepy'